<a href="https://www.kaggle.com/code/chanoncharuchinda/hotel-booking-analysis-using-pyspark?scriptVersionId=132678975" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


# Hotel Booking Analysis

The aim of this project is to analyze the [Hotel booking demand](https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand) dataset. <br>
Throughout this notebook, I will highlight the question that I am trying to solve. For this notebook, I will be using [PySpark](https://spark.apache.org/docs/latest/api/python/#:~:text=PySpark%20is%20the%20Python%20API,for%20interactively%20analyzing%20your%20data.) to perform queries. 

Last semester, I took a Big Data Analysis course, which is where I was introduced to Apache Spark. <br>
At this moment,  I am in the process of revising this concept; hence, I choose PySpark for this notebook. 

If you have any suggestion, please let me know!

### Installation
In order to use PySpark on [Kaggle](https://www.kaggle.com/), it must be installed by typing the following command `!pip install pyspark` into one of the cell. Can add option `-q` which mean quite to not show the downloading progress. 

#### By: Chanon Charuchinda

In [1]:
!pip install pyspark -q

# Need to clean

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * # A better practice would be importing only necessary functions (like comments below)
# from pyspark.sql.functions import col, sum, count, avg, round
# from pyspark.sql.functions import length, desc, expr, sum, count
# import pyspark

spark = SparkSession.builder.appName("HotelAnalysis").master("local[*]").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 15:15:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
hotel_bookings_path = "/kaggle/input/hotel-booking-demand/hotel_bookings.csv"
hotelBooking = spark.read.csv(hotel_bookings_path, inferSchema=True, header=True) # DataFrame type

In [4]:
# Show the schema of the data
hotelBooking.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

## Possible Analysis Questions

1. What is the overall cancellation rate for hotel bookings?
2. How does the cancellation rate vary across different hotels?
3. Are there any specific months or seasons with higher cancellation rates?
4. How does lead time (number of days between booking and arrival) affect the cancellation rate?
5. What is the average length of stay (in nights) for hotel bookings?
6. How do the number of adults, children, and babies affect the cancellation rate?
7. Which market segments contribute the most to hotel bookings?
8. How does the distribution channel impact the cancellation rate?
9. What is the distribution of booking changes made by customers?
10. How does the deposit type influence the cancellation behavior of customers?
11. Are there any particular countries that have a higher cancellation rate?
12. How do previous cancellations and previous bookings not canceled affect the current cancellation rate?
13. Which types of rooms (reserved and assigned) have the highest cancellation rates?
14. How does the customer type relate to the cancellation rate?
15. Does the average daily rate (ADR) differ for canceled and non-canceled bookings?


# Distribution of Price that Guests Paid for a Room

#### Main columns that will be used in this segment:

* `adr`: Average Daily Rate as defined by dividing the sum of all lodging transactions by the total number of staying nights

* `arrival_date_year`: Year of arrival date

* `arrival_date_month`: Month of arrival date

* `stays_in_weekend_nights`: Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel

* `stays_in_week_nights`: Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel


#### Computed Features
* Variables: 
> 1. *Total Number of Night* (including weekends and weekdays): **num_nights**
> 2. *Total Number of Guests* (including Adults, Children, and Babies): **num_guests**
> 3. *Average Daily Rate* (adr): **adr**

* Calculations:
> 1. *Total Booking Cost*: **total_booking_cost = num_nights * adr**
> 2. *Average Booking Cost per Guest*: **avr_booking_cost_per_guest = total_booking_cost / num_guests**


In [5]:
hotelBooking_index = (
    hotelBooking
    .select("*")
    .withColumn("index", monotonically_increasing_id())
)

# After cleaning, remove this import statement
import pyspark.sql.types as T

features = ["index","hotel","is_canceled","arrival_date_year",
            "arrival_date_month","adr","num_nights","num_guests"]

hotelBookingCost = (
    hotelBooking_index
    .withColumn("num_guests", (col("adults") + col("children") + col("babies")).cast(T.IntegerType()))
    .withColumn("num_nights", (col("stays_in_weekend_nights") + col("stays_in_week_nights").cast(T.IntegerType())))
).select(*features)

# Removed rows with average daily rate of 0.0
# Alternative Approach (for dealing with 0.0 value in adr column) in the following cell 
hotelBookingCostFiltered = (
    hotelBookingCost
    .filter((col("adr") != 0) & (col("num_nights") != 0) & (col("num_guests") != 0))
)

summaryHotelBookingCost = (
    hotelBookingCostFiltered
    .withColumn("total_booking_cost", round(col("num_nights") * col("adr"),2))
    .withColumn("avr_booking_per_guest", round(col("total_booking_cost") / col("num_guests"),2))
)

summaryHotelBookingCost.show()

+-----+------------+-----------+-----------------+------------------+------+----------+----------+------------------+---------------------+
|index|       hotel|is_canceled|arrival_date_year|arrival_date_month|   adr|num_nights|num_guests|total_booking_cost|avr_booking_per_guest|
+-----+------------+-----------+-----------------+------------------+------+----------+----------+------------------+---------------------+
|    2|Resort Hotel|          0|             2015|              July|  75.0|         1|         1|              75.0|                 75.0|
|    3|Resort Hotel|          0|             2015|              July|  75.0|         1|         1|              75.0|                 75.0|
|    4|Resort Hotel|          0|             2015|              July|  98.0|         2|         2|             196.0|                 98.0|
|    5|Resort Hotel|          0|             2015|              July|  98.0|         2|         2|             196.0|                 98.0|
|    6|Resort Hotel|

In [6]:
# TODO: Explore MORE!
# Alternative: Fill in the 0.0 with average daily rate of the hotel (we can also factor in month to capture the possible seasonal effects on the price)
# For instance, in the tourist peak season, the hotel would want to charge their guests more to earn more money

# Find the average adr of each month
adrByMonth = (
    hotelBooking_index
    .select("arrival_date_year","arrival_date_month", "adr")
)

adrByMonthFiltered = (
    adrByMonth
    .filter(col("adr") != 0)
)

avrByYearMonth = (
    adrByMonthFiltered
    .groupBy("arrival_date_year","arrival_date_month")
    .agg(
        round(avg(col("adr")),2).alias("average_adr")
    )
).orderBy("arrival_date_year","average_adr")

print("Computing Average Daily Rate for each pair of year and month: ")
avrByYearMonth.filter(col("arrival_date_year") == "2015").show()

# LEFT JOIN: hotelBookingCost and avrByYearMonth
# conditions := condition for performing LEFT JOIN operation
conditions = (
    (hotelBookingCost["arrival_date_year"] == avrByYearMonth["arrival_date_year"]) &
    (hotelBookingCost["arrival_date_month"] == avrByYearMonth["arrival_date_month"])
)

final_features = ["hotel","is_canceled","num_nights","num_guests","finalized_adr"]

joinedHotelBookingAdr = (
    hotelBookingCost
    .filter((col("num_nights") != 0) & (col("num_guests") != 0))
    .join(avrByYearMonth, conditions ,"left")
    .withColumn("finalized_adr", when(col("adr").cast(T.IntegerType()) == 0, col("average_adr"))
                                .otherwise(col("adr")))
).select(*final_features)

print("Finalized table with adjusted average daily rate: ")
joinedHotelBookingAdr.show()

summaryHotelBookingCostALT = (
    joinedHotelBookingAdr
    .withColumn("total_booking_cost", round(col("num_nights") * col("finalized_adr"),2))
    .withColumn("avr_booking_per_guest", round(col("total_booking_cost") / col("num_guests"),2))
)

print("Summary: ")
summaryHotelBookingCostALT.show()

Computing Average Daily Rate for each pair of year and month: 


+-----------------+------------------+-----------+
|arrival_date_year|arrival_date_month|average_adr|
+-----------------+------------------+-----------+
|             2015|          November|      62.34|
|             2015|          December|      77.39|
|             2015|           October|      81.07|
|             2015|         September|      96.81|
|             2015|              July|     100.07|
|             2015|            August|     108.58|
+-----------------+------------------+-----------+

Finalized table with adjusted average daily rate: 


+------------+-----------+----------+----------+-------------+
|       hotel|is_canceled|num_nights|num_guests|finalized_adr|
+------------+-----------+----------+----------+-------------+
|Resort Hotel|          0|         1|         1|         75.0|
|Resort Hotel|          0|         1|         1|         75.0|
|Resort Hotel|          0|         2|         2|         98.0|
|Resort Hotel|          0|         2|         2|         98.0|
|Resort Hotel|          0|         2|         2|        107.0|
|Resort Hotel|          0|         2|         2|        103.0|
|Resort Hotel|          1|         3|         2|         82.0|
|Resort Hotel|          1|         3|         2|        105.5|
|Resort Hotel|          1|         4|         2|        123.0|
|Resort Hotel|          0|         4|         2|        145.0|
|Resort Hotel|          0|         4|         2|         97.0|
|Resort Hotel|          0|         4|         3|       154.77|
|Resort Hotel|          0|         4|         2|       

#### Explore how much does the hotel earned and lost

In [7]:
hotelRevenue = (
    summaryHotelBookingCostALT
    .groupBy("hotel")
    .agg(
        sum("total_booking_cost").alias("expected_revenue"),
        sum(expr(f"CASE WHEN is_canceled = 1 THEN total_booking_cost ELSE 0 END")).alias("revenue_lost"),
        (col("expected_revenue") - col("revenue_lost")).alias("revenue"),
        (col("revenue_lost") * 100 / col("expected_revenue")).alias("revenue_lost_percentage")
    )
    .select(
        col("hotel"),
        format_number(col("expected_revenue"), 2).alias("expected_revenue"),
        format_number(col("revenue"),2).alias("revenue"),
        format_number(col("revenue_lost"), 2).alias("revenue_lost"),
        round(col("revenue_lost_percentage"),2).alias("revenue_lost_percentage")
    )
)

print("Hotel Revenue Summary: ")
hotelRevenue.show()

Hotel Revenue Summary: 


+------------+----------------+-------------+-------------+-----------------------+
|       hotel|expected_revenue|      revenue| revenue_lost|revenue_lost_percentage|
+------------+----------------+-------------+-------------+-----------------------+
|Resort Hotel|   17,559,414.91|11,696,738.17| 5,862,676.74|                  33.39|
|  City Hotel|   25,416,514.35|14,516,135.68|10,900,378.67|                  42.89|
+------------+----------------+-------------+-------------+-----------------------+



# Reserved and Assigned Room Analysis

#### Main columns that will be used in this segment:
`reserved_room_type`: Code of room type reserved. Code is presented instead of designation for anonymity reasons.

`assigned_room_type`: Code for the type of room assigned to the booking. Sometimes the assigned room type differs from the reserved room type due to hotel operation reasons (e.g. overbooking) or by customer request. Code is presented instead of designation for anonymity reasons.

#### Objective:
How much does not getting the room that the guests asked for effect the cancellation rate at each hotel? <br>
We can address this question by comparing the two columns `reserved_room_types` and `assigned_room_type`. <br>

```ruby
CREATE new column "isRequestedRoom"

FOR each rows in hotelBooking:
    # Get requested room
    IF reserved_room_types.value == assigned_room_type:
    THEN isRequestedRoom = 0

    # Not getting requested room
    ELSE: isRequestedRoom = 1 
```

In [8]:
room_features = ["hotel","is_canceled","reserved_room_type","assigned_room_type","isRequestedRoom"]

reservedRoom = (
    hotelBooking
    .withColumn("isRequestedRoom", when(col("reserved_room_type") == col("assigned_room_type"), 1)
                                  .otherwise(0))
).select(*room_features)

hotelReservedRoom = (
    reservedRoom
    .groupBy("hotel")
    .agg(
        round((sum(col("isRequestedRoom")) * 100 / count(col("hotel"))),2).alias("getting_requested_room_per")
    )
)
print("Describe the percentage of guest that get the room they requested: ")
hotelReservedRoom.show()

# Out of all the canceled reservation, how many reservation does the guest didn't get the requested room
print("Relationship between Booking Cancellation and Not getting assigned room")
canceledReservationRoomDiff = (
    reservedRoom
    .filter(col("is_canceled") == 1)
    .groupBy("hotel")
    .agg(
        sum(col("isRequestedRoom")).alias("total_not_assinged_room"),
        count("hotel").alias("total_cancellation"),
        round((sum(col("isRequestedRoom")) * 100 / count("hotel")),2).alias("canceled_not_requested_room_per")
    )
)

canceledReservationRoomDiff.show()

Describe the percentage of guest that get the room they requested: 


+------------+--------------------------+
|       hotel|getting_requested_room_per|
+------------+--------------------------+
|Resort Hotel|                     80.72|
|  City Hotel|                     90.93|
+------------+--------------------------+

Relationship between Booking Cancellation and Not getting assigned room
+------------+-----------------------+------------------+-------------------------------+
|       hotel|total_not_assinged_room|total_cancellation|canceled_not_requested_room_per|
+------------+-----------------------+------------------+-------------------------------+
|Resort Hotel|                  10738|             11122|                          96.55|
|  City Hotel|                  32684|             33102|                          98.74|
+------------+-----------------------+------------------+-------------------------------+



# Returning Customers Analysis

#### Main columns that will be used in this segment:

`is_repeated_guest`: Value indicating if the booking name was from a repeated guest (1) or not (0)

`previous_cancellations`: Number of previous bookings that were cancelled by the customer prior to the current booking

`previous_bookings_not_canceled`: Number of previous bookings not cancelled by the customer prior to the current booking

In [9]:
# Return Guests at the hotel
# What exactly is the meaning of previous_cancellations and previous_bookings_not_canceled

guestReturningHist = (
    hotelBooking
    .select("hotel","is_canceled","is_repeated_guest","previous_cancellations","previous_bookings_not_canceled")
    .groupBy("hotel")
    .agg(
        round((sum("is_canceled") * 100 / count("hotel")),2).alias("cancellation_rate"), 
        round((sum("is_repeated_guest") * 100 / count("hotel")),2).alias("returnGuest_rate"), 
        sum("previous_cancellations").alias("total_prev_canceled"), 
        sum("previous_bookings_not_canceled").alias("total_prev_not_canceled"), 
        count("hotel").alias("total_booking")
    )
)
    

guestReturningHist.show()

+------------+-----------------+----------------+-------------------+-----------------------+-------------+
|       hotel|cancellation_rate|returnGuest_rate|total_prev_canceled|total_prev_not_canceled|total_booking|
+------------+-----------------+----------------+-------------------+-----------------------+-------------+
|Resort Hotel|            27.76|            4.44|               4075|                   5867|        40060|
|  City Hotel|            41.73|            2.56|               6326|                  10501|        79330|
+------------+-----------------+----------------+-------------------+-----------------------+-------------+



# Relation between Agent and Customer Type

In [10]:
# Another approach. Create temp view and use SQL queries 
hotelBooking.createOrReplaceTempView("hotel_booking")


queries = """
    select 
        customer_type,
        sum(is_canceled) as total_canceled
    from hotel_booking
    group by customer_type
    order by total_canceled desc
"""

customerTypeSQL = spark.sql(queries)

print("Customer Type and Total Cancellation")
customerTypeSQL.show()

23/06/07 15:15:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Customer Type and Total Cancellation
+---------------+--------------+
|  customer_type|total_canceled|
+---------------+--------------+
|      Transient|         36514|
|Transient-Party|          6389|
|       Contract|          1262|
|          Group|            59|
+---------------+--------------+



In [11]:
# Customer Type, Agent, and Company
from pyspark.sql.types import IntegerType

customerAgentRelation = hotelBooking \
    .select("hotel","is_canceled","customer_type",col("agent").cast(IntegerType()), "previous_cancellations") \
    .filter(col("agent").isNotNull()) \
    .groupBy("agent","hotel","customer_type") \
    .agg(
        sum("is_canceled").alias("total_cancellation"), \
        count("customer_type").alias("customer_number"), \
        round((col("total_cancellation") * 100 / col("customer_number")),2).alias("cancellation_rate"), \
        sum("previous_cancellations").alias("total_prev_cancellation") # Recheck this 
    ).orderBy(asc("agent"), "hotel", "customer_type")

print("Agent and Customer Type Relation Analysis: ")
# customerAgentRelation.show(customerAgentRelation.count())

customerAgentRelation.show(20)

Agent and Customer Type Relation Analysis: 


+-----+------------+---------------+------------------+---------------+-----------------+-----------------------+
|agent|       hotel|  customer_type|total_cancellation|customer_number|cancellation_rate|total_prev_cancellation|
+-----+------------+---------------+------------------+---------------+-----------------+-----------------------+
|    1|  City Hotel|       Contract|               700|            700|            100.0|                    701|
|    1|  City Hotel|          Group|                 0|              2|              0.0|                      0|
|    1|  City Hotel|      Transient|              2974|           3021|            98.44|                    612|
|    1|  City Hotel|Transient-Party|              1556|           3414|            45.58|                   1138|
|    1|Resort Hotel|      Transient|                50|             54|            92.59|                      0|
|    2|  City Hotel|      Transient|                 3|              8|             37.5

In [12]:
# Overall cancellation rate (and in percentage) of this dataset
hotelBooking.select(expr("avg(is_canceled) as average_cancellation")).withColumn("avg_cancel_per", col("average_cancellation") * 100).show()

+--------------------+------------------+
|average_cancellation|    avg_cancel_per|
+--------------------+------------------+
| 0.37041628277075134|37.041628277075134|
+--------------------+------------------+



In [13]:
# Average Booking Volumes for each Agent
# Average Booking cancellation for each hotel when the booking is done via an agent
hotelBooking.select("hotel","is_canceled",col("agent").cast(IntegerType())).filter(col("agent").isNotNull()) \
.groupBy("hotel").avg("is_canceled").show()

+------------+------------------+
|       hotel|  avg(is_canceled)|
+------------+------------------+
|Resort Hotel|0.3045744246648457|
|  City Hotel|0.4282784870574025|
+------------+------------------+



In [14]:
hotelBooking.select("hotel","is_canceled",col("agent").cast(IntegerType())).filter(col("agent").isNotNull()) \
.groupBy("agent","hotel").agg(count("agent").alias("total_transaction")).orderBy("agent").show()

+-----+------------+-----------------+
|agent|       hotel|total_transaction|
+-----+------------+-----------------+
|    1|Resort Hotel|               54|
|    1|  City Hotel|             7137|
|    2|Resort Hotel|              102|
|    2|  City Hotel|               60|
|    3|Resort Hotel|               28|
|    3|  City Hotel|             1308|
|    4|  City Hotel|               47|
|    5|Resort Hotel|              228|
|    5|  City Hotel|              102|
|    6|Resort Hotel|              607|
|    6|  City Hotel|             2683|
|    7|  City Hotel|             3539|
|    8|Resort Hotel|              278|
|    8|  City Hotel|             1236|
|    9|Resort Hotel|                6|
|    9|  City Hotel|            31955|
|   10|Resort Hotel|               13|
|   10|  City Hotel|              247|
|   11|  City Hotel|              290|
|   11|Resort Hotel|              105|
+-----+------------+-----------------+
only showing top 20 rows



In [15]:
# Agent, Booking Cancellation, and Track record of previous cancellation
hotelBooking.select(col("agent").cast(IntegerType()),"is_canceled","previous_cancellations").filter(col("agent").isNotNull()) \
.orderBy("agent").show()

+-----+-----------+----------------------+
|agent|is_canceled|previous_cancellations|
+-----+-----------+----------------------+
|    1|          1|                     1|
|    1|          0|                     0|
|    1|          1|                     0|
|    1|          0|                     0|
|    1|          1|                     1|
|    1|          0|                     0|
|    1|          1|                     0|
|    1|          0|                     0|
|    1|          1|                     1|
|    1|          0|                     0|
|    1|          1|                     0|
|    1|          0|                     0|
|    1|          1|                     1|
|    1|          0|                     0|
|    1|          1|                     0|
|    1|          0|                     0|
|    1|          1|                     1|
|    1|          0|                     0|
|    1|          1|                     0|
|    1|          0|                     0|
+-----+----

# Market Segment

In [16]:
# QUESTION: Cancelation by market Segment
bookingMarketSegment = hotelBooking \
    .select("hotel","is_canceled","market_segment","deposit_type","customer_type")\
    .groupBy("market_segment")\
    .agg(
        sum("is_canceled").alias("total_canceled"),\
        count("market_segment").alias("total_booking")
    ).withColumn("cancellation_percentage", round(col("total_canceled") * 100 / col("total_booking"),2)) \
    .orderBy(desc("cancellation_percentage"))

bookingMarketSegment.show()


# Just showing that there are multiple ways of doing this 

print("Using SQL Queries")
# Register the DataFrame as a temporary view
hotelBooking.createOrReplaceTempView("hotel_booking")

# Execute the SQL query
bookingMarketSegment_SQL = spark.sql("""
    SELECT market_segment,
           SUM(is_canceled) AS total_canceled,
           COUNT(market_segment) AS total_booking,
           ROUND(SUM(is_canceled) * 100 / COUNT(market_segment), 2) AS cancellation_percentage
    FROM hotel_booking
    GROUP BY market_segment
    ORDER BY cancellation_percentage DESC
""")

bookingMarketSegment_SQL.show()

+--------------+--------------+-------------+-----------------------+
|market_segment|total_canceled|total_booking|cancellation_percentage|
+--------------+--------------+-------------+-----------------------+
|     Undefined|             2|            2|                  100.0|
|        Groups|         12097|        19811|                  61.06|
|     Online TA|         20739|        56477|                  36.72|
| Offline TA/TO|          8311|        24219|                  34.32|
|      Aviation|            52|          237|                  21.94|
|     Corporate|           992|         5295|                  18.73|
|        Direct|          1934|        12606|                  15.34|
| Complementary|            97|          743|                  13.06|
+--------------+--------------+-------------+-----------------------+

Using SQL Queries
+--------------+--------------+-------------+-----------------------+
|market_segment|total_canceled|total_booking|cancellation_percentage|
+

# Country

In [17]:
# QUESTION: Countries distribution and their cancellation

bookingCountry = hotelBooking.select("hotel","is_canceled","country")

canceledBookingCountry = bookingCountry\
    .groupBy("country")\
    .agg(
        sum("is_canceled").alias("total_cancellation"), \
        count("country").alias("total_booking"), \
    ).withColumn("cancellation_percentage", round(col("total_cancellation") * 100 / col("total_booking"),2)) \
    .orderBy(desc("total_cancellation"))

canceledBookingCountry.show()

+-------+------------------+-------------+-----------------------+
|country|total_cancellation|total_booking|cancellation_percentage|
+-------+------------------+-------------+-----------------------+
|    PRT|             27519|        48590|                  56.64|
|    GBR|              2453|        12129|                  20.22|
|    ESP|              2177|         8568|                  25.41|
|    FRA|              1934|        10415|                  18.57|
|    ITA|              1333|         3766|                   35.4|
|    DEU|              1218|         7287|                  16.71|
|    IRL|               832|         3375|                  24.65|
|    BRA|               830|         2224|                  37.32|
|    USA|               501|         2097|                  23.89|
|    BEL|               474|         2342|                  20.24|
|    CHN|               462|          999|                  46.25|
|    CHE|               428|         1730|                  24

# Guests (Adults, Children, and Babies) Distribution

In [18]:
# QUESTION: Adualt, Chldren, and Babies at the Hotel

guestDistribution = hotelBooking \
    .select("hotel","is_canceled","adults","children","babies") \
    .withColumn("canceld_string", expr("case when is_canceled = 0 then 'No' else 'Yes' end")) \
    
guestDistribution.show()

+------------+-----------+------+--------+------+--------------+
|       hotel|is_canceled|adults|children|babies|canceld_string|
+------------+-----------+------+--------+------+--------------+
|Resort Hotel|          0|     2|       0|     0|            No|
|Resort Hotel|          0|     2|       0|     0|            No|
|Resort Hotel|          0|     1|       0|     0|            No|
|Resort Hotel|          0|     1|       0|     0|            No|
|Resort Hotel|          0|     2|       0|     0|            No|
|Resort Hotel|          0|     2|       0|     0|            No|
|Resort Hotel|          0|     2|       0|     0|            No|
|Resort Hotel|          0|     2|       0|     0|            No|
|Resort Hotel|          1|     2|       0|     0|           Yes|
|Resort Hotel|          1|     2|       0|     0|           Yes|
|Resort Hotel|          1|     2|       0|     0|           Yes|
|Resort Hotel|          0|     2|       0|     0|            No|
|Resort Hotel|          0

In [19]:
# ADULT
adultBooking = hotelBooking \
    .select("adults","children","babies","agent","company","is_canceled") \
    .withColumn("via_agent", expr("case when agent >=0 then 1 else 0 end")) \
    .withColumn("via_company", expr("case when company >=0 then 1 else 0 end")) \
    .groupBy("adults") \
    .agg( 
        sum("children").alias("num_children"), \
        sum("babies").alias("num_babies"), \
        sum("is_canceled").alias("total_cancelation"), \
        sum("via_agent").alias("total_via_agent"), \
        sum("via_company").alias("total_via_company"), \
        count("adults").alias("total_booking"), \
    ).orderBy(col("adults"))

adultBooking.show()

+------+------------+----------+-----------------+---------------+-----------------+-------------+
|adults|num_children|num_babies|total_cancelation|total_via_agent|total_via_company|total_booking|
+------+------------+----------+-----------------+---------------+-----------------+-------------+
|     0|       453.0|         3|              109|            326|               31|          403|
|     1|       605.0|        22|             6674|          15717|             4747|        23027|
|     2|     10778.0|       911|            35258|          81163|             1955|        89680|
|     3|       565.0|        12|             2151|           5789|               63|         6202|
|     4|         2.0|         1|               16|             50|                1|           62|
|     5|         0.0|         0|                2|              0|                0|            2|
|     6|         0.0|         0|                1|              0|                0|            1|
|    10|  

# Booking by Months Analysis

In [20]:
# QUESTION: Explore the behavior of number of stays and variation in season (only success booking)
hotelNights = hotelBooking \
    .select("hotel","is_canceled","arrival_date_month","stays_in_weekend_nights","stays_in_week_nights") \
    .filter( (col("is_canceled") == 0) & (col("stays_in_weekend_nights") != 0) & (col("stays_in_week_nights") != 0)) \
    .withColumn("total_night", col("stays_in_weekend_nights") + col("stays_in_week_nights")) \

hotelNights.show()

+------------+-----------+------------------+-----------------------+--------------------+-----------+
|       hotel|is_canceled|arrival_date_month|stays_in_weekend_nights|stays_in_week_nights|total_night|
+------------+-----------+------------------+-----------------------+--------------------+-----------+
|Resort Hotel|          0|              July|                      1|                   4|          5|
|Resort Hotel|          0|              July|                      2|                   4|          6|
|Resort Hotel|          0|              July|                      2|                   4|          6|
|Resort Hotel|          0|              July|                      2|                   4|          6|
|Resort Hotel|          0|              July|                      2|                   5|          7|
|Resort Hotel|          0|              July|                      2|                   5|          7|
|Resort Hotel|          0|              July|                      2|    

In [21]:
# Average night at each hotel
hotelNights.groupBy("hotel").avg().show()

+------------+----------------+----------------------------+-------------------------+------------------+
|       hotel|avg(is_canceled)|avg(stays_in_weekend_nights)|avg(stays_in_week_nights)|  avg(total_night)|
+------------+----------------+----------------------------+-------------------------+------------------+
|Resort Hotel|             0.0|           1.894015503875969|        4.147968992248062| 6.041984496124031|
|  City Hotel|             0.0|          1.5249860413176997|        2.455285687697748|3.9802717290154477|
+------------+----------------+----------------------------+-------------------------+------------------+



In [22]:
month_order = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

summaryHotelNights = hotelNights \
    .groupBy("hotel","arrival_date_month") \
    .agg(
        sum("total_night").alias("total_nights_by_month"), \
        round(avg("total_night")).alias("average_night_by_month") \
    ).withColumn("month_index", expr("array_position(array({}), arrival_date_month)".format(",".join("'{}'".format(month) for month in month_order)))) \
    .orderBy("hotel","month_index")

summaryHotelNights.show(summaryHotelNights.count())

+------------+------------------+---------------------+----------------------+-----------+
|       hotel|arrival_date_month|total_nights_by_month|average_night_by_month|month_index|
+------------+------------------+---------------------+----------------------+-----------+
|  City Hotel|           January|                 3825|                   4.0|          1|
|  City Hotel|          February|                 5778|                   4.0|          2|
|  City Hotel|             March|                 7934|                   4.0|          3|
|  City Hotel|             April|                 8179|                   4.0|          4|
|  City Hotel|               May|                 7425|                   4.0|          5|
|  City Hotel|              June|                 7321|                   4.0|          6|
|  City Hotel|              July|                 9976|                   4.0|          7|
|  City Hotel|            August|                11385|                   4.0|          8|

In [23]:
# Average Night at Each Hotel
summaryHotelNights.groupBy("hotel").agg(round(avg("average_night_by_month")).alias("average_nights_per_year")).show()

+------------+-----------------------+
|       hotel|average_nights_per_year|
+------------+-----------------------+
|  City Hotel|                    4.0|
|Resort Hotel|                    6.0|
+------------+-----------------------+



In [24]:
# QUESTION: Investigate the data about agents for Resort Hotel (similar analysis can be done for City Hotel as well)

agentSummary = hotelBooking \
    .select(col("agent").cast(IntegerType()), "is_canceled", "hotel") \
    .filter((col("agent") >= 0) & (col("hotel") == "Resort Hotel")) \
    .groupBy("agent")\
    .agg(
        sum("is_canceled").alias("total_canceled"), \
        count("agent").alias("total_booking"), \
    ) \
    .withColumn("canceled_rate", expr("round(total_canceled * 100 / total_booking, 2)")) \
    .orderBy(desc("total_canceled"))

agentSummary.show()

+-----+--------------+-------------+-------------+
|agent|total_canceled|total_booking|canceled_rate|
+-----+--------------+-------------+-------------+
|  240|          5483|        13905|        39.43|
|  250|           513|         2869|        17.88|
|  242|           260|          779|        33.38|
|   96|           244|          537|        45.44|
|  241|           236|         1721|        13.71|
|  134|           177|          287|        61.67|
|   68|           165|          211|         78.2|
|  314|           163|          927|        17.58|
|  298|           142|          472|        30.08|
|   38|           132|          236|        55.93|
|  273|           121|          349|        34.67|
|  315|           102|          256|        39.84|
|  208|            95|          173|        54.91|
|   15|            93|          242|        38.43|
|  248|            83|          131|        63.36|
|   40|            82|         1002|         8.18|
|  177|            78|         

# Overall Successed and Failed Booking Analysis

In [25]:
# QUESTION: For each Hotel, what is the number of successful booking, canceled booking, and average cancelation 

hotelBookingSummary = hotelBooking \
    .select("hotel","is_canceled") \
    .groupBy("hotel") \
    .agg( \
        sum("is_canceled").alias("total_canceled"), \
        count("hotel").alias("total_booking"), \
        round((avg("is_canceled") * 100),2).alias("cancelation_ratio")
    )

hotelBookingSummary.show()

+------------+--------------+-------------+-----------------+
|       hotel|total_canceled|total_booking|cancelation_ratio|
+------------+--------------+-------------+-----------------+
|Resort Hotel|         11122|        40060|            27.76|
|  City Hotel|         33102|        79330|            41.73|
+------------+--------------+-------------+-----------------+



## Thank you for checking out this notebook

Please leave comments and suggestion for improvement.

TODO: Add conclusion to each analysis topics, More questions to be answered

DISCLAIMER: This is work in progress. I will be updating this notebook really soon!

# Clean the Notebook

In [26]:
# COMMAND: Install Pyspark for this notebook
# !pip install pyspark -q

In [27]:
# # IMPORT necessary libraries
# from pyspark.sql import SparkSession
# import pyspark.sql.functions as F
# import pyspark.sql.types as T

# # Create Spark Session
# spark = (
#     SparkSession
#     .builder
#     .appName("HotelAnalysis")
#     .master("local[*]")
#     .getOrCreate()
# )

Hote Booking Demand Dataset (schema): <br>
`root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullable = true)
 |-- booking_changes: integer (nullable = true)
 |-- deposit_type: string (nullable = true)
 |-- agent: string (nullable = true)
 |-- company: string (nullable = true)
 |-- days_in_waiting_list: integer (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- adr: double (nullable = true)
 |-- required_car_parking_spaces: integer (nullable = true)
 |-- total_of_special_requests: integer (nullable = true)
 |-- reservation_status: string (nullable = true)
 |-- reservation_status_date: date (nullable = true)`

In [28]:
# hotelSchema = (
#     T.StructType()
#     .add("hotel", T.StringType(), nullable = True)
#     .add("is_canceled", T.IntegerType(), nullable = True)
#     .add("lead_time", T.IntegerType(), nullable = True)
#     .add("arrival_date_year", T.IntegerType(), nullable = True)
#     .add("arrival_date_month", T.StringType(), nullable = True)
# )

# hotelSchema.fieldNames()